# Kaggle Submission

This is a notebook that just formats data for a kaggle submission with the results of previous notebooks.

In [110]:
import pickle
import pandas as pd
import numpy as np
import patsy
import re

Previous results are loaded to be used later.

In [111]:
name = 'lessout'

In [112]:
fileObj = open('./pickles/formula'+name+'.pkl', 'rb')
formula = pickle.load(fileObj)
fileObj.close()

In [113]:
fileObj = open('./pickles/model'+name+'.pkl', 'rb')
model = pickle.load(fileObj)
fileObj.close()

In [114]:
fileObj = open('./pickles/scaler'+name+'.pkl', 'rb')
scaler = pickle.load(fileObj)
fileObj.close()

In [115]:
fileObj = open('./pickles/columns'+name+'.pkl', 'rb')
columns = pickle.load(fileObj)
fileObj.close()

The formula is first replaced so that the target is removed.

In [116]:
formula = formula[12:]

In [117]:
test = pd.read_csv('./data/test.csv', index_col=0)

In [118]:
test.columns = test.columns.map(lambda x: x.replace(' ', ''))\
                    .map(lambda x: re.sub('^2', 'Seco', x))\
                    .map(lambda x: re.sub('^1', 'Fir', x))\
                    .map(lambda x: re.sub('^3', 'Third', x))\
                    .map(lambda x: x.replace('/', ''))

The formatting procedure from the previous two steps is used, though it is only important that columns that were previously used are selected and the same model is imported.

In [119]:
test = test.drop('PID', axis=1).fillna(0)

In [120]:
test.MSSubClass = test.MSSubClass.astype('category')
test.YrSold = test.YrSold.astype('category')

In [121]:
x1 = patsy.dmatrix(formula, test)

There are different dummy columns created due to the difference in test and train csv's.  Because it was not dealt with before, we now set the columns between the two datasets equivalent.  The information exclusive to the test set will be lost, unfortunately.

In [122]:
test_cols = set(x1.design_info.column_names)

In [123]:
orig_cols = set(columns)

In [124]:
diff_cols1 = orig_cols.difference(test_cols)
diff_cols2 = test_cols.difference(orig_cols)

In [125]:
x1 = pd.DataFrame(x1, columns=x1.design_info.column_names)

In [126]:
len(diff_cols1)

30

In [127]:
len(diff_cols2)

11

In [128]:
for column in diff_cols1:
    x1[column] = 0
    
for column in diff_cols2:
    x1.drop(column, axis=1, inplace=True)

In [129]:
x1 = x1[columns]
x1 = scaler.transform(x1)

In [130]:
y1 = pd.DataFrame(model.predict(x1), index=test.index, columns=['SalePrice'])

In [131]:
y1.to_csv('./scores/predictions'+name+'.csv')

In [132]:
y1

,SalePrice
Id,
2658,153895.684732
2718,157890.182894
2414,213869.765458
1989,101780.337738
625,177509.560929
333,86434.483243
1327,106436.055349
858,143054.094752
95,184891.094625
